In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import mlflow
import os

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, f
from hyperopt.pyll import scope


In [2]:

mlflow.autolog()
mlflow_dir = "/home/sam/Documents/projects/practice/mlops/crop-recomendation"
# mlflow.set_tracking_uri(f"sqlite:///{os.path.join(mlflow_dir, 'mlflow.db')}")
# mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_tracking_uri(os.path.join(mlflow_dir, "mlruns"))

mlflow.set_experiment("crop-recommendation")

2024/07/18 11:16:52 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


2024/07/18 11:16:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


<Experiment: artifact_location='/home/sam/Documents/projects/practice/mlops/crop-recomendation/mlruns/885963556934924601', creation_time=1721251298940, experiment_id='885963556934924601', last_update_time=1721251298940, lifecycle_stage='active', name='crop-recommendation', tags={}>

In [3]:
def read(filename: str) -> pd.DataFrame:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    label_encoder = LabelEncoder()

    # Fit and transform the data
    df['label'] = label_encoder.fit_transform(df['label'])

    with open('../models/label_encoder.bin', 'wb') as f_out:
        pickle.dump(label_encoder, f_out)
    
    return df

In [4]:
def preprocess(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:


    X = df.drop(['label', 'P'], axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

    # Step 4: Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    
    return X_train, X_test, y_train, y_test

In [5]:
def train(X_train: np.ndarray, y_train: np.ndarray) -> LogisticRegression:

    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    


    with open('../models/log_reg.bin', 'wb') as f_out:
        pickle.dump(lr, f_out)


    return lr

    

In [6]:
def evaluate(X_test: np.ndarray, y_test: np.ndarray, lr_model=None) -> Tuple[int, int]:

    if lr_model == None:
        with open('../models/log_reg.bin', 'rb') as file:
            lr_model = pickle.load(file)

    y_pred = lr_model.predict(X_test)

    acc_score = accuracy_score(y_test, y_pred)
    confu_mat = confusion_matrix(y_test, y_pred)

    print(f'Accuracy: {acc_score}')
    print(f'Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}')

    return acc_score


In [7]:
def predict(
        feature1: float, feature2: float, 
        feature3: float, feature4: float, 
        feature5: float, feature6: float
        ) -> None:

    with open('../models/label_encoder.bin', 'rb') as f_in:
        label_encoder = pickle.load(f_in)

    with open('../models/log_reg.bin', 'rb') as f_in:
        lr = pickle.load(f_in)

    # Prepare new data for prediction
    new_data = np.array([[feature1, feature2, feature3, feature4, feature5, feature6]])

    # Make a prediction
    prediction = lr.predict(new_data)
    print("Prediction (encoded):", prediction)

    # Reverse the label encoding
    original_label = label_encoder.inverse_transform([round(prediction[0])])
    print("Prediction (original label):", original_label[0])


In [9]:
df = read("../data/Crop_recommendation.csv")

In [10]:
X_train, X_test, y_train, y_test = preprocess(df)

In [19]:
def objective(params: dict):
    with mlflow.start_run():
        mlflow.set_tag("model", "LogisticRegression")
        mlflow.log_params(params)

        # Instantiate the Logistic Regression model
        lr = LogisticRegression(
            C=params['C'],
            # penalty=params['penalty'],
            solver=params['solver'],
            max_iter=params['max_iter'],
            random_state=params['random_state'],
        )

        # Fit the model
        lr.fit(X_train, y_train)

        # Evaluate the model on the test set
        y_pred = lr.predict(X_test)
        acc_score = accuracy_score(y_test, y_pred)
        mlflow.log_metric("test_acc_score", acc_score)

    return -acc_score

    # return {'accuracy_score': -acc_score, 'status': STATUS_OK}

# Define the search space
search_space = {
    'C': hp.loguniform('C', -5, 5),
    # 'penalty': hp.choice('penalty', [None, 'l2']),
    'solver': hp.choice('solver', ['liblinear', 'lbfgs']),
    'max_iter': scope.int(hp.quniform('max_iter', 100, 1000, 100)),
    'random_state': 42
}

# Optimize the hyperparameters
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)



  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2024/07/18 11:42:12 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



  2%|▏         | 1/50 [00:02<02:02,  2.49s/trial, best loss: -0.7318181818181818]

2024/07/18 11:42:14 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



  4%|▍         | 2/50 [00:04<01:49,  2.28s/trial, best loss: -0.7863636363636364]

...

 96%|█████████▌| 48/50 [01:49<00:05,  2.58s/trial, best loss: -0.9704545454545455]

2024/07/18 11:44:02 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'

/home/sam/Documents/projects/practice/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 98%|█████████▊| 49/50 [01:52<00:02,  2.71s/trial, best loss: -0.9704545454545455]

2024/07/18 11:44:05 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



100%|██████████| 50/50 [01:55<00:00,  2.31s/trial, best loss: -0.9704545454545455]


In [1]:
best_result

NameError: name 'best_result' is not defined

In [45]:
lr = train(X_train, y_train, X_test, y_test)

2024/07/18 00:21:58 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


In [84]:
acc_score= evaluate(X_test, y_test, lr)

Accuracy: 0.9386363636363636
Confusion Matrix:
 [[26  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 19  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 20  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 23  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 24  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 21  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 19  0  0  4  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 22  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2  0  1 15  0  0  0  0

In [89]:
predict(10,	81,	10.879744, 2.002744, 3.502985, 2.935536)

Prediction (encoded): [7]
Prediction (original label): grapes


In [ ]:
df.sort_values()